# Session

In [ ]:
#| default_exp session

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import csv

import os
import re
from pathlib import Path
from typing import Tuple, List, Union, Optional, Callable, Type, Iterable, no_type_check, Sequence
from IPython import display

import pandas as pd
import os

import logging
logger = logging.getLogger(__name__)

from spannerlib.utils import checkLogs,serialize_df_values,get_base_file_path
from spannerlib.grammar import parse_spannerlog,reconstruct
from spannerlib.span import Span
from spannerlib.engine import (
    Engine,_infer_relation_schema,
    Var,
    FreeVar,
    RelationDefinition,
    Relation,
    IEFunction,
    IERelation,
    Rule,
    pretty,
)

from spannerlib.micro_passes import (
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames,
    check_referenced_paths_exist,
    check_referenced_vars_exist,
    relations_to_dataclasses,
    verify_referenced_relations,
    rules_to_dataclasses,
    consistent_free_var_types_in_rule,
    check_rule_safety,
    assignments_to_name_val_tuple,
    execute_statement,
)


In [ ]:
#| export
#| hide


from spannerlib.ie_func.json_path import JsonPath, JsonPathFull
from spannerlib.ie_func.nlp import (Tokenize, SSplit, POS, Lemma, NER, EntityMentions, CleanXML, Parse, DepParse, Coref, OpenIE, KBP, Quote, Sentiment, TrueCase)
from spannerlib.ie_func.python_regex import PYRGX, PYRGX_STRING
from spannerlib.ie_func.rust_spanner_regex import RGX, RGX_STRING, RGX_FROM_FILE, RGX_STRING_FROM_FILE


# ordered by rgx, json, nlp, etc.
PREDEFINED_IE_FUNCS = [PYRGX, PYRGX_STRING, RGX, RGX_STRING, RGX_FROM_FILE, RGX_STRING_FROM_FILE,
                       JsonPath, JsonPathFull,
                       Tokenize, SSplit, POS, Lemma, NER, EntityMentions, CleanXML, Parse, DepParse, Coref, OpenIE, KBP, Quote, Sentiment,
                       TrueCase]





## Session class

In [ ]:
#| export
class Session():
    def __init__(self):
        
        self.pass_stack = [
            convert_primitive_values_to_objects,
            remove_new_lines_from_strings,
            CheckReservedRelationNames('spanner_'),
            check_referenced_paths_exist,
            check_referenced_vars_exist,
            relations_to_dataclasses,
            verify_referenced_relations,
            rules_to_dataclasses,
            consistent_free_var_types_in_rule,
            check_rule_safety,
            assignments_to_name_val_tuple
        ]
        self.engine=Engine()

        # TODO define all the default ie funcs

    def get_pass_stack(self):
        """
        @return: the current pass stack.
        """
        return self._pass_stack.copy()

    def set_pass_stack(self,
        user_stack #  a user supplied pass stack
        ): 
        """
        Sets a new pass stack instead of the current one.
        """
        if type(user_stack) is not list:
            raise TypeError('user stack should be a list of passes')
        self._pass_stack = user_stack.copy()

    def _display_result(self,result):
        if result is None:
            pass
        elif isinstance(result,pd.DataFrame):
            display.display(result)
        else:
            print(result)
    
    def register(self,name,func,in_schema,out_schema):
        #TODO add option for outschema to be a callable that gets the input and the output and confirms if the output is valid
        ie_func_obj = IEFunction(name=name,func=func,in_schema=in_schema,out_schema=out_schema)
        self.engine.set_ie_function(ie_func_obj)

    def export(self,code,display_results=False):
        #TODO reconstruct the code for each statement using lark,reconstruct so we can print the query string together with the result
        statements = parse_spannerlog(code,split_statements=True)
        results = []
        for statement_nx,statement_lark in statements:
            ast = statement_nx
            for pass_ in self.pass_stack:
                try:
                    pass_(ast,self.engine)
                except Exception as e:
                    raise Exception(
                        f"During semantic checks for statement \n\"{reconstruct(statement_lark)}\"\n"
                        f"in pass {pass_} the following exception was raised:\n{e}\n"
                        ).with_traceback(e.__traceback__)

            try:
                result = execute_statement(ast,self.engine)
            except Exception as e:
                raise Exception(
                    f"During execution of statement \n\"{reconstruct(statement_lark)}\n\""
                    f"the following exception was raised:\n{e}\n"
                    ).with_traceback(e.__traceback__)
            results.append(result)
            if display_results:
                self._display_result(result)
        return results[-1]

    def import_relation(self,name:str,data:Union[str,Path,pd.DataFrame],delim:str = None):
        """Imports a relation into the current session, either from a dataframe or from a csv file."""
        if isinstance(data, (Path,str)):
            csv_file_name = Path(data)
            if not csv_file_name.is_file():
                raise IOError("csv file does not exist")
            if os.stat(csv_file_name).st_size == 0:
                raise IOError("csv file is empty")
            data = pd.read_csv(csv_file_name, delimiter=delim)

        first_row = list(data.iloc[0,:])
        schema = _infer_relation_schema(first_row)
        rel_def = RelationDefinition(name,schema)
        self.engine.set_relation(rel_def,data)
        self.engine.add_facts(name,data)
        
    def print_rules(self):
        for rule in self.engine.rules_to_ids.keys():
            print(rule)
    def remove_rule(self,rule:str):
        self.engine.del_rule(rule)
    def remove_relation(self,relation:str):
        self.engine.del_relation(relation)
    def clear(self):
        self.engine = Engine()

    def get_all_functions(self):
        return self.engine.ie_functions.copy()

## Examples

In [ ]:
commands = """
    new Parent(str, str)
    Parent("Sam", "Noah")
    Parent("Noah", "Austin")
    Parent("Austin", "Stephen")

    GrandParent(G, C) <- Parent(G, M), Parent(M, C)
    ?GrandParent(X, "Austin")
    """
session = Session()
res = session.export(commands,display_results=True)
assert serialize_df_values(res) == {('Sam',)}

# check that rerunning query gives the same result
res = session.export('?GrandParent(X, "Austin")')
assert serialize_df_values(res) == {('Sam',)}

,X
0,Sam


In [ ]:
def length(string: str) -> Iterable[int]:
        yield len(string)

sess = Session()
sess.register('length',length,[str],[int])
res = sess.export('''
    new string(str)
    string("a")
    string("d")
    string("a")
    string("ab")
    string("abc")
    string("abcd")

    string_length(Str, Len) <- string(Str), Length(Str) -> (Len)
    ?string_length(Str, Len)
''')
#TODO from here, add the rest of the examples from the old notebook and start moving the failed ones to tests in the respective passes/execution
res

Exception: During semantic checks for statement 
"string_length(Str,Len)<-string(Str),Length(Str)->(Len)"
in pass <function consistent_free_var_types_in_rule at 0x120c87600> the following exception was raised:
'NoneType' object has no attribute 'in_schema'


## Tests

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     